# Taller 7

### SQL

##### Analítica Computacional para la Toma de Decisiones

---

|     Nombres      |      Apellidos       |     Login     |  Codigo   |
| :--------------: | :------------------: | :-----------: | :-------: |
|     Santiago     | Gonzalez Montealegre | s.gonzalez35  | 202012274 |
| Juliana Carolina |  Cardenas Barragan   | jc.cardenasb1 | 202011683 |

---

---

---

## 1. Consultas Básicas

---

---

### 1 

Cree un Jupyter notebook, el cual servirá como documento de trabajo y entrega. Incluya todas las respuestas a las consultas solicitadas en el notebook.

---

### 2
Conéctese al mismo servidor usado en las diapositivas, pero en este caso a la base de datos `taller_uni`.

In [1]:
import psycopg2
import pandas as pd

In [2]:
connection = psycopg2.connect(
    host="db-actd23-1.c2ejf18x6i7a.us-east-1.rds.amazonaws.com",
    port='5432',
    user="postgres",
    dbname="taller_uni",
    password="postgres",
)

Se definirán funciónes con el fin de facilitar las consultas y no tener que repetir código. Además, facilitar la interpretación de las consultas.

In [3]:
# ------------------------------------------------
# Mostrar el resultado como un pandas DataFrame
# ------------------------------------------------

# Ejecutar la query
def execute(query: str, cursor: (psycopg2.extensions.cursor)):
    cursor.execute(query)
    result = cursor.fetchall()
    return result

# Hallar los atributos
def getColumnNames(cursor):
    columns = [i[0] for i in cursor.description]
    return columns

# Mostrar como DataFrame
def pdsql(query: str, connection: (psycopg2.extensions.connection)):
    try:
        cursor = connection.cursor()
        result = execute(query, cursor)
        columns = getColumnNames(cursor)
        df = pd.DataFrame(result, columns=columns)
        return df
    except Exception:
        connection.commit()
        return "Query Error"

---

### 3
Liste todas las tablas en el esquema público.

In [4]:
query = """
SELECT * 
FROM pg_catalog.pg_tables 
WHERE schemaname='public'
;"""
df = pdsql(query, connection)
df[["tablename"]]

,tablename
0,unid_acad
1,curso
2,instructor
3,grupo
4,dicta


---

### 4
Liste el nombre de todas las unidades académicas.

In [5]:
query = """
SELECT *
FROM unid_acad
;"""
df = pdsql(query, connection)
df[["nombre_unid"]]

,nombre_unid
0,IIND
1,ECON
2,BIOL


---

### 5
Liste los nombres de todas las unidades académicas a las que están asociados los profesores. Elimine duplicados con `distinct` antes del atributo de interés (nombre de la unidad).


In [6]:
query = """
SELECT DISTINCT unid_acad.nombre_unid
FROM unid_acad
NATURAL INNER JOIN instructor
;"""
df = pdsql(query, connection)
df

,nombre_unid
0,IIND
1,ECON


---

### 6
Para cada tabla liste 5 registros usando `limit` al final de la consulta.

In [7]:
query = """
SELECT *
FROM unid_acad
LIMIT 5
;"""
df = pdsql(query, connection)
df

,nombre_unid,edificio,presupuesto
0,IIND,ML,100.00
1,ECON,W,800.00
2,BIOL,A,500.00


In [8]:
query = """
SELECT *
FROM curso
LIMIT 5
;"""
df = pdsql(query, connection)
df

,curso_cod,nombre,nombre_unid,creditos
0,000,probyest1,IIND,3
1,001,anadec,IIND,3
2,002,optimizacion,IIND,3
3,003,micro1,ECON,3


In [9]:
query = """
SELECT *
FROM instructor
LIMIT 5
;"""
df = pdsql(query, connection)
df

,inst_id,nombre,nombre_unid,salario
0,1,Pedro Perez,IIND,100.00
1,2,Ana Alferez,IIND,200.00
2,3,Camila Cifuentes,ECON,500.00
3,4,Reinaldo Rondon,IIND,300.00


In [10]:
query = """
SELECT *
FROM grupo
LIMIT 5
;"""
df = pdsql(query, connection)
df

,curso_cod,grupo_cod,semestre,ano
0,000,1,1,2023
1,001,1,2,2023
2,002,1,1,2023
3,002,2,1,2023


In [11]:
query = """
SELECT *
FROM dicta
LIMIT 5
;"""
df = pdsql(query, connection)
df

,inst_id,curso_cod,grupo_cod,semestre,ano
0,1,000,1,1,2023
1,2,001,1,2,2023
2,3,002,1,1,2023
3,4,002,1,1,2023


---

### 7
Realice una consulta que retorne un listado de los `instructores` con su nombre, ID y el doble de su salario.

In [12]:
query = """
SELECT nombre, inst_id, 2 * salario as salarioX2
FROM instructor
;"""
df = pdsql(query, connection)
df

,nombre,inst_id,salariox2
0,Pedro Perez,1,200.00
1,Ana Alferez,2,400.00
2,Camila Cifuentes,3,1000.00
3,Reinaldo Rondon,4,600.00


---

### 8
Realice una consulta para obtener todos los cursos ofrecidos por la unidad `'IIND'`.

In [13]:
query = """
SELECT *
FROM curso
WHERE nombre_unid = 'IIND'
;"""
df = pdsql(query, connection)
df

,curso_cod,nombre,nombre_unid,creditos
0,000,probyest1,IIND,3
1,001,anadec,IIND,3
2,002,optimizacion,IIND,3


---

### 9
Liste todos los nombres de los instructores con el edificio donde queda ubicada su respectiva unidad. Use un cross join.

In [14]:
query = """
SELECT nombre, edificio
FROM instructor, unid_acad
WHERE instructor.nombre_unid = unid_acad.nombre_unid
;"""
df = pdsql(query, connection)
df

,nombre,edificio
0,Pedro Perez,ML
1,Ana Alferez,ML
2,Camila Cifuentes,W
3,Reinaldo Rondon,ML


---

### 10
Liste todos los instructores con sus clases. Use un cross join.

In [15]:
query = """
SELECT instructor.nombre, curso.nombre
FROM instructor, dicta, curso
WHERE instructor.inst_id = dicta.inst_id
AND dicta.curso_cod = curso.curso_cod
;"""
df = pdsql(query, connection)
df

,nombre,nombre
0,Pedro Perez,probyest1
1,Ana Alferez,anadec
2,Camila Cifuentes,optimizacion
3,Reinaldo Rondon,optimizacion


---

### 11
Liste todos los instructores con sus clases. Use un natural join.

In [16]:
query = """
SELECT instructor.nombre, curso.nombre
FROM instructor
NATURAL INNER JOIN dicta
INNER JOIN curso ON dicta.curso_cod = curso.curso_cod
;"""
df = pdsql(query, connection)
df

,nombre,nombre
0,Pedro Perez,probyest1
1,Ana Alferez,anadec
2,Camila Cifuentes,optimizacion
3,Reinaldo Rondon,optimizacion


---

### 12
Liste todos los IDs de los instructores con los códigos de los cursos que dictan. Use un natural join.

In [17]:
query = """
SELECT instructor.inst_id, curso.curso_cod
FROM instructor
NATURAL INNER JOIN dicta
JOIN curso USING (curso_cod)
;"""
df = pdsql(query, connection)
df

,inst_id,curso_cod
0,1,000
1,2,001
2,3,002
3,4,002


Alternativamente se puede usar `JOIN USING` para hacer un join que use solo algunos atributos comunes, no todos. Por ejemplo, para listar todos los IDs de los instructores con los códigos de los cursos que dictan podemos ejecutar el siguiente comando:
```SQL
SELECT inst_ID , curso_cod
FROM (instructor NATURAL JOIN dicta) 
JOIN curso USING (curso_cod);
```

La clausula `AS` se puede utilizar para renombrar los atributos en la relación que se obtiene como resultado. Por ejemplo, en el siguiente comando cambiamos el nombre del primer atributo seleccionado
```SQL
SELECT nombre AS nombre_inst, curso_cod 
FROM instructor, dicta
WHERE instructor.inst_id = dicta.inst_id;
```

La cláusula `AS` también se puede utilizar para renombrar las relaciones consideradas. Por ejemplo, el comando anterior lo podríamos escribir como
```SQL
SELECT nombre, curso_cod
FROM instructor as I, dicta AS D
WHERE I.inst_ID = D.inst_ID;
```

Esto puede ser u´til cuando se requiere comparar una relaci´on consigo misma. Por ejemplo, si se quiere determinar los cursos que tienen al menos tantos cr´editos como al menos un curso de IIND, ejecutar´ıamos el comando
```SQL
SELECT DISTINCT B.nombre 
FROM curso AS B, curso AS D
WHERE B.creditos > D.creditos and B.nombre_unid = 'IIND';
```
Estos nombres para las relaciones se conocen como alias.

---

### 13
Realice una consulta en la que compare una relación consigo misma (diferente a curso).

In [18]:
# Programa con presupuesto mayor a aquellos programas en el edificio ML
query = """
SELECT A.nombre_unid
FROM unid_acad AS A, unid_acad AS B
WHERE A.presupuesto > B.presupuesto
AND B.edificio = 'ML'
;"""
df = pdsql(query, connection)
df

,nombre_unid
0,ECON
1,BIOL


---

---

## 2. Más Consultas

---

---

### 1
Liste todos los cursos con una `'a'` en su nombre.

In [19]:
query = """
SELECT *
FROM curso
WHERE nombre LIKE '%a%'
;"""
df = pdsql(query, connection)
df

,curso_cod,nombre,nombre_unid,creditos
0,001,anadec,IIND,3
1,002,optimizacion,IIND,3


---

### 2
Liste todos los instructores cuya unidad empieza con `'I'` de ingeniería.

1. Para estos instructores liste los códigos de todos los cursos que dictaron.
1. Para estos instructores liste los códigos y nombres de todos los cursos que
dictaron, así como el año y semestre.

In [20]:
query = """
SELECT instructor.nombre, curso.curso_cod, curso.nombre, grupo.ano, grupo.semestre
FROM instructor
INNER JOIN dicta ON instructor.inst_id = dicta.inst_id
INNER JOIN curso ON dicta.curso_cod = curso.curso_cod
INNER JOIN grupo ON curso.curso_cod = grupo.curso_cod
WHERE instructor.nombre_unid LIKE 'I%'
;"""
df = pdsql(query, connection)
df

,nombre,curso_cod,nombre,ano,semestre
0,Pedro Perez,000,probyest1,2023,1
1,Ana Alferez,001,anadec,2023,2
2,Reinaldo Rondon,002,optimizacion,2023,1
3,Reinaldo Rondon,002,optimizacion,2023,1


---

### 3
Determine el número promedio de créditos en los cursos ofrecidos por `'IIND'`.


In [21]:
query = """
SELECT AVG(creditos) AS prom_creditos
FROM curso
WHERE nombre_unid = 'IIND'
;"""
df = pdsql(query, connection)
df

,prom_creditos
0,3.0000000000000000


---

### 4
Liste el número promedio créditos en los cursos ofrecidos por cada unidad.

1. Ordene sus resultados por el nombre de la unidad.
1. Filtre sus resultados para incluir solamente las unidades que ofrezcan curso con
un promedio de máximo 4 créditos.

In [22]:
query = """
SELECT nombre_unid, AVG(creditos) prom_creditos
FROM curso
GROUP BY nombre_unid
HAVING AVG(creditos) <= 4
ORDER BY nombre_unid
;"""
df = pdsql(query, connection)
df

,nombre_unid,prom_creditos
0,ECON,3.0000000000000000
1,IIND,3.0000000000000000


---

### 5
Liste el salario promedio de los profesores de cada unidad académica, así como el salario mínimo y máximo. 

Su relación debe tener 4 atributos con nombres `'nombre_unid'`, `'prom_salario'`, `'min_salario'`, `'max_salario'`, en ese orden.

In [23]:
query = """
SELECT nombre_unid, AVG(salario) prom_salario, MIN(salario) min_salario, MAX(salario) max_salario
FROM instructor
GROUP BY nombre_unid
;"""
df = pdsql(query, connection)
df

,nombre_unid,prom_salario,min_salario,max_salario
0,IIND,200.0000000000000000,100.00,300.00
1,ECON,500.0000000000000000,500.00,500.00


---

---

## 3. Otra Base de Datos: world

---

---

### 1
Conéctese al mismo endpoint de las secciones anteriores, pero a la base de datos
`world`.

In [24]:
connection = psycopg2.connect(
    host="db-actd23-1.c2ejf18x6i7a.us-east-1.rds.amazonaws.com",
    port='5432',
    user="postgres",
    dbname="world",
    password="postgres",
)

Actualizamos nuestra función, para ser usada en la nueva base de datos

In [25]:
# ------------------------------------------------
# Mostrar el resultado como un pandas DataFrame
# ------------------------------------------------

# Ejecutar la query
def execute(query: str, cursor: (psycopg2.extensions.cursor)):
    cursor.execute(query)
    result = cursor.fetchall()
    return result

# Hallar los atributos
def getColumnNames(cursor):
    columns = [i[0] for i in cursor.description]
    return columns

# Mostrar como DataFrame
def pdsql(query: str, connection: (psycopg2.extensions.connection)):
    try:
        cursor = connection.cursor()
        result = execute(query, cursor)
        columns = getColumnNames(cursor)
        df = pd.DataFrame(result, columns=columns)
        return df
    
    except Exception:
        connection.commit()
        return "Query Error"

In [26]:
query = """
SELECT * 
FROM pg_catalog.pg_tables 
WHERE schemaname='public'
;"""
df = pdsql(query, connection)
df[["tablename"]]

,tablename
0,city
1,country
2,country_language


---

### 2
Esta base tiene 3 tablas: `city`, `country` y `country_language`. Con esta información realicemos algunas consultas.

Revisemos por ejemplos las primeras 5 filas de la tabla `city`:

In [27]:
query = """
SELECT *
FROM city
LIMIT 5
;"""
df = pdsql(query, connection)
df

,id,name,country_code,district,population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200


Si en algún momento se genera un error, la conexión queda bloqueada con una
transacción que no terminó. Para descartar la transacción actual y volver a intentar
ejecute un commit sobre la conexión.

```js
connection.commit()
```

---

### 3
Cuente ahora el número de registros en cada tabla.

In [28]:
query = """
SELECT COUNT(*) city_rows
FROM city
;"""
df = pdsql(query, connection)
df

,city_rows
0,4079


In [29]:
query = """
SELECT COUNT(*) country_rows
FROM country
;"""
df = pdsql(query, connection)
df

,country_rows
0,239


In [30]:
query = """
SELECT COUNT(*) country_language_rows
FROM country_language
;"""
df = pdsql(query, connection)
df

,country_language_rows
0,984


Para facilitar las consultas próximas se traerán las primeras filas de cada una de las relaciones.

In [31]:
query = """
SELECT *
FROM city
LIMIT 5
;"""
df = pdsql(query, connection)
df

,id,name,country_code,district,population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200


In [32]:
query = """
SELECT *
FROM country
LIMIT 5
;"""
df = pdsql(query, connection)
df

,code,name,continent,region,surface_area,indep_year,population,life_expectancy,gnp,gnp_old,local_name,government_form,head_of_state,capital,code2
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.00,None,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1,AF
1,NLD,Netherlands,Europe,Western Europe,41526.0,1581.0,15864000,78.3,371362.00,360478.00,Nederland,Constitutional Monarchy,Beatrix,5,NL
2,ANT,Netherlands Antilles,North America,Caribbean,800.0,NaN,217000,74.7,1941.00,None,Nederlandse Antillen,Nonmetropolitan Territory of The Netherlands,Beatrix,33,AN
3,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.00,2500.00,Shqipëria,Republic,Rexhep Mejdani,34,AL
4,DZA,Algeria,Africa,Northern Africa,2381741.0,1962.0,31471000,69.7,49982.00,46966.00,Al-Jazair/Algérie,Republic,Abdelaziz Bouteflika,35,DZ


In [33]:
query = """
SELECT *
FROM country_language
LIMIT 5
;"""
df = pdsql(query, connection)
df

,country_code,language,is_official,percentage
0,AFG,Pashto,True,52.4
1,NLD,Dutch,True,95.6
2,ANT,Papiamento,True,86.2
3,ALB,Albaniana,True,97.9
4,DZA,Arabic,True,86.0


---

### 4
Usando la tabla country, determine los continentes que aparecen en la tabla.

In [34]:
query = """
SELECT DISTINCT continent
FROM country
;"""
df = pdsql(query, connection)
df

,continent
0,Asia
1,South America
2,North America
3,Oceania
4,Antarctica
5,Africa
6,Europe


---

### 5
Liste los nombres y códigos de los países en el continente Europe.

In [35]:
query = """
SELECT code, name
FROM country
WHERE continent = 'Europe'
;"""
df = pdsql(query, connection)
df

,code,name
0,NLD,Netherlands
1,ALB,Albania
2,AND,Andorra
3,BEL,Belgium
4,BIH,Bosnia and Herzegovina
5,GBR,United Kingdom
6,BGR,Bulgaria
7,ESP,Spain
8,FRO,Faroe Islands
9,GIB,Gibraltar


---

### 6
Determine los 5 países con mayor población de Europa.

In [36]:
query = """
SELECT *
FROM country
WHERE continent = 'Europe'
ORDER BY population DESC
LIMIT 5
;"""
df = pdsql(query, connection)
df

,code,name,continent,region,surface_area,indep_year,population,life_expectancy,gnp,gnp_old,local_name,government_form,head_of_state,capital,code2
0,RUS,Russian Federation,Europe,Eastern Europe,17075400.0,1991,146934000,67.2,276608.00,442989.00,Rossija,Federal Republic,Vladimir Putin,3580,RU
1,DEU,Germany,Europe,Western Europe,357022.0,1955,82164700,77.4,2133367.00,2102826.00,Deutschland,Federal Republic,Johannes Rau,3068,DE
2,GBR,United Kingdom,Europe,British Islands,242900.0,1066,59623400,77.7,1378330.00,1296830.00,United Kingdom,Constitutional Monarchy,Elisabeth II,456,GB
3,FRA,France,Europe,Western Europe,551500.0,843,59225700,78.8,1424285.00,1392448.00,France,Republic,Jacques Chirac,2974,FR
4,ITA,Italy,Europe,Southern Europe,301316.0,1861,57680000,79.0,1161755.00,1145372.00,Italia,Republic,Carlo Azeglio Ciampi,1464,IT


---

### 7
Determine la población promedio de los países de Europa.

In [37]:
query = """
SELECT AVG(population) avg_pop_europe
FROM country
WHERE continent = 'Europe'
;"""
df = pdsql(query, connection)
df

,avg_pop_europe
0,15871186.956521739130


---

### 8
Liste los países que tienen una población superior al promedio del continente. 

Para cada país incluya nombre y población.

In [38]:
query = """
SELECT country.name, country.population
FROM country
INNER JOIN (
    SELECT continent, AVG(population) as avg_pop
    FROM country
    GROUP BY continent
    ) AS avg ON country.continent = avg.continent
WHERE country.population > avg.avg_pop
;"""
df = pdsql(query, connection)
df

,name,population
0,Algeria,31471000
1,Argentina,37032000
2,Australia,18886000
3,Bangladesh,129155000
4,Brazil,170115000
5,United Kingdom,59623400
6,Egypt,68470000
7,Spain,39441700
8,South Africa,40377000
9,Ethiopia,62565000


---

### 9
Determine los tipos de gobierno existentes en Europa.

In [39]:
query = """
SELECT DISTINCT government_form
FROM country
WHERE continent = 'Europe'
;"""
df = pdsql(query, connection)
df

,government_form
0,Republic
1,Dependent Territory of Norway
2,Parliamentary Coprincipality
3,Part of Denmark
4,Constitutional Monarchy
5,Federal Republic
6,"Constitutional Monarchy, Federation"
7,Dependent Territory of the UK
8,Independent Church State
9,Federation


---

### 10
Determine los países europeos que tienen un tipo de gobierno entre los tres más comunes en el continente. 

Para cada país incluya nombre y población.

In [40]:
query = """
SELECT name, population
FROM country
WHERE government_form IN (
    SELECT government_form
    FROM country
    WHERE continent = 'Europe'
    GROUP BY government_form
    ORDER BY COUNT(*) DESC
    LIMIT 3
    )
AND continent = 'Europe'
;"""
df = pdsql(query, connection)
df

,name,population
0,Netherlands,15864000
1,Albania,3401200
2,Bosnia and Herzegovina,3972000
3,United Kingdom,59623400
4,Bulgaria,8190900
5,Spain,39441700
6,Ireland,3775100
7,Iceland,279000
8,Italy,57680000
9,Austria,8091800


---

### 11
Liste cada país con su capital, población total, población de la capital y fracción de población que vive en la capital. 

Reporte el top 20 de los países con mayor y menor fracción de población en la capital. 

Por motivos de legibilidad, se espera que la fracción se muestre como un número con 4 posiciones decimales. 

Hint: considere el uso de CAST AS y ROUND.

- Top 20 países con mayor fracción de población en la capital.

In [41]:
query = """
SELECT country.name, city.name, country.population, city.population, ROUND(((city.population) * 1.0 / country.population), 4) AS frac_pop_capital
FROM country
INNER JOIN city ON country.capital = city.id
ORDER BY frac_pop_capital DESC
LIMIT 20
;"""
df = pdsql(query, connection)
df

,name,name,population,population,frac_pop_capital
0,Singapore,Singapore,3567000,4017733,1.1264
1,Gibraltar,Gibraltar,25000,27025,1.0810
2,Macao,Macao,473000,437500,0.9249
3,Pitcairn,Adamstown,50,42,0.8400
4,Saint Pierre and Miquelon,Saint-Pierre,7000,5808,0.8297
5,Falkland Islands,Stanley,2000,1636,0.8180
6,Palau,Koror,19000,12000,0.6316
7,Djibouti,Djibouti,638000,383000,0.6003
8,Cook Islands,Avarua,20000,11900,0.5950
9,Qatar,Doha,599000,355000,0.5927


- Top 20 países con menor fracción de población en la capital.

In [42]:
query = """
SELECT country.name, city.name, country.population, city.population, ROUND(((city.population) * 1.0 / country.population), 4) AS frac_pop_capital
FROM country
INNER JOIN city ON country.capital = city.id
ORDER BY frac_pop_capital ASC
LIMIT 20
;"""
df = pdsql(query, connection)
df

,name,name,population,population,frac_pop_capital
0,India,New Delhi,1013662000,301297,0.0003
1,United States,Washington,278357000,572059,0.0021
2,Nigeria,Abuja,111506000,350100,0.0031
3,Pakistan,Islamabad,156483000,524500,0.0034
4,Tanzania,Dodoma,33517000,189000,0.0056
5,China,Peking,1277558000,7472000,0.0058
6,Guam,Agaña,168000,1139,0.0068
7,Côte dIvoire,Yamoussoukro,14786000,130000,0.0088
8,Bhutan,Thimphu,2124000,22000,0.0104
9,Netherlands Antilles,Willemstad,217000,2345,0.0108


- Se solicitaba el top 20 de los países con mayor y menor fracción de población en la capital, sin embargo, no era claro si era top 20 de mayor y top 20 de menor para finalmente unirlos en una sola tabla resultado.

- Por lo tanto, se tomó esta interpretación.

In [43]:
query = """
(
    SELECT country.name, city.name, country.population, city.population, ROUND(((city.population) * 1.0 / country.population), 4) AS frac_pop_capital
    FROM country
    INNER JOIN city ON country.capital = city.id
    ORDER BY frac_pop_capital DESC
    LIMIT 20
) UNION (
    SELECT country.name, city.name, country.population, city.population, ROUND(((city.population) * 1.0 / country.population), 4) AS frac_pop_capital
    FROM country
    INNER JOIN city ON country.capital = city.id
    ORDER BY frac_pop_capital ASC
    LIMIT 20
)
ORDER BY frac_pop_capital DESC
;"""
df = pdsql(query, connection)
df

,name,name,population,population,frac_pop_capital
0,Singapore,Singapore,3567000,4017733,1.1264
1,Gibraltar,Gibraltar,25000,27025,1.0810
2,Macao,Macao,473000,437500,0.9249
3,Pitcairn,Adamstown,50,42,0.8400
4,Saint Pierre and Miquelon,Saint-Pierre,7000,5808,0.8297
5,Falkland Islands,Stanley,2000,1636,0.8180
6,Palau,Koror,19000,12000,0.6316
7,Djibouti,Djibouti,638000,383000,0.6003
8,Cook Islands,Avarua,20000,11900,0.5950
9,Qatar,Doha,599000,355000,0.5927


---

### 12
Genere una lista de los países que hablan inglés como idioma oficial. 

Para cada país cuyo idioma oficial es el inglés, liste la fracción de la población que habla inglés y ordene de acuerdo de menor a mayor.

In [44]:
query = """
SELECT country.name, country_language.percentage
FROM country
INNER JOIN country_language ON country.code = country_language.country_code
WHERE country_language.language = 'English'
AND country_language.is_official = TRUE
ORDER BY country_language.percentage ASC
;"""
df = pdsql(query, connection)
df

,name,percentage
0,Montserrat,0.0
1,Niue,0.0
2,Antigua and Barbuda,0.0
3,United States Minor Outlying Islands,0.0
4,Barbados,0.0
5,Anguilla,0.0
6,Tuvalu,0.0
7,Turks and Caicos Islands,0.0
8,"Virgin Islands, British",0.0
9,Cayman Islands,0.0


---

### 13
Determine los 5 idiomas más hablados por número de países y por población total que lo habla.

Se interpreta la pregunta como dos consultas distintas:

1. Determinar los 5 idiomas más hablados por el número de países que lo hablan.
1. Determinar los 5 idiomas más hablados por la población que lo habla.

In [45]:
query = """
SELECT country_language.language, COUNT(country.code) AS num_countries_speaks
FROM country
INNER JOIN country_language ON country.code = country_language.country_code
GROUP BY country_language.language
ORDER BY COUNT(country.code) DESC
LIMIT 5
;"""
df = pdsql(query, connection)
df

,language,num_countries_speaks
0,English,60
1,Arabic,33
2,Spanish,28
3,French,25
4,Chinese,19


In [46]:
query = """
SELECT A.language, SUM(A.pop_speaks) pop_speaks
FROM (
    SELECT country.code, country.population, country_language.language, country_language.percentage, CAST(country.population * country_language.percentage / 100 AS INTEGER) AS pop_speaks
    FROM country
    INNER JOIN country_language ON country.code = country_language.country_code
    ) AS A
GROUP BY A.language
ORDER BY SUM(A.pop_speaks) DESC
LIMIT 5
;"""
df = pdsql(query, connection)
df

,language,pop_speaks
0,Chinese,1191843539
1,Hindi,405633085
2,Spanish,355029462
3,English,347077859
4,Arabic,233839239
